In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
from os import path


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
html_text = requests.get('https://www.compari.ro/placi-video-c3142/',headers=headers)
soup = BeautifulSoup(html_text.text,'lxml')
Placi_Video = soup.find_all('div',class_='col-lg-6 col-md-6 col-sm-6 col-xs-8 top-center')

Nume = []
Lista_Pret = []
Link_curat = []
Titlu = []
today = date.today()
data_curr = []
Chipset =[]
Ventilatoare = []

lista_link = []
counter = 0
for Placa in Placi_Video:
    link=Placa.find('a',href = True)
    lista_link.append(link['href'])
    

In [2]:
from datetime import date, datetime

for i in lista_link:
    html_link2 = requests.get(i,headers=headers)
    soup2 = BeautifulSoup(html_link2.text,'lxml')
    
    Pret = soup2.find('span',class_='price').text.replace('RON','').replace(',','.').replace('de la','').replace(' ','')
    
    Surse = soup2.find_all('div',class_='optoffer device-desktop')
    for Sursa in Surse:
        Pret_Surse = Sursa.find('div',class_='row-price').text.replace('RON','').replace(',','.').replace('de la','').replace(' ','')
        
        if Pret == Pret_Surse:
                Link_Sursa = Sursa.find('a',href=True)
                Link_curat.append(Link_Sursa['href'])
                Titlu.append(Sursa.find('div',class_='col-button button-box').text.replace('spre magazin »',''))
                Nume.append(soup2.find('h1',class_='visible-xs').text.replace('\n','').replace('Placa video','').replace('   ',''))
                Lista_Pret.append(soup2.find('span',class_='price').text.replace('RON','').replace(',','.').replace('de la','').replace(' ',''))
                data_curr.append(today.strftime("%d/%m/%Y"))
                Chip = soup2.find('td',text = 'Familie chipset video')
                if Chip is not None:    
                    Chipset.append(soup2.find('td',text = 'Familie chipset video').find_next_sibling('td').text.replace(' ',''))
                else:
                    Chipset.append('NaN')
                Ventilator = soup2.find('td',text = 'Numar ventilatoare')
                if Ventilator is not None:
                    Ventilatoare.append(soup2.find('td',text = 'Numar ventilatoare').find_next_sibling('td').text.replace('buc',''))
                else:
                    Ventilatoare.append(0)
                
        else:
            Link_Sursa = 'Nu s a gasit'
    

for i in range(len(Lista_Pret)):
    Lista_Pret[i] = float(Lista_Pret[i])
    Ventilatoare[i] = int(Ventilatoare[i])
    

In [3]:
import pandas as pd

Daily_GPA_df = pd.DataFrame()
Daily_GPA_df['Data curenta'] = data_curr
Daily_GPA_df['Denumire Produs'] = Nume
Daily_GPA_df['Pret LEI'] = Lista_Pret
Daily_GPA_df['Site'] = Titlu
Daily_GPA_df['Sursa'] = Link_curat
Daily_GPA_df['Chipset'] = Chipset
Daily_GPA_df['Nr Ventilatoare'] = Ventilatoare


In [4]:
if path.exists('AnalizaPlaci.csv'):
    print('Nu s-a creat')
else:
    Daily_GPA_df.to_csv(r'AnalizaPlaci.csv',index=False)
    print('S-a creat')

Nu s-a creat


In [5]:
from datetime import date
data_azi = date.today()
GPU_Comparator_df2 = pd.read_csv('AnalizaPlaci.csv')
if Daily_GPA_df['Data curenta'].iloc[0] != GPU_Comparator_df2['Data curenta'].iloc[-1]:
    Daily_GPA_df.to_csv('AnalizaPlaci.csv', mode='a', header=False,index=False)
    df3 = pd.read_csv('AnalizaPlaci.csv')
    df3.to_csv('AnalizaPlaci2.csv',index = False)
    print(f"Adaugare completa {data_azi}")
else:
    print(f'''Ai adaugat deja azi: {data_azi}, incearca maine''')


Adaugare completa 2022-02-09


In [6]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
from pprint import pprint

Filter_df = pd.read_csv('AnalizaPlaci2.csv',index_col=None)
Filter_df = Filter_df.drop('Ventilatoare',axis=1)
Filter_df['Data curenta'] = pd.to_datetime(Filter_df['Data curenta'],dayfirst = True)
df3 =Filter_df.groupby(['Data curenta','Denumire Produs']).mean().unstack().reset_index()
#df3.iloc[:,0] = pd.to_datetime(df3.iloc[:,0])

lista1= []
listanp = df3.columns.to_numpy()
lista1 = np.append(lista1,listanp[0][0])
listanp = np.delete(listanp,0)
for i in listanp:
    lista1 = np.append(lista1, i[1])

df3.columns = lista1
df3.to_csv("VideoCardsPrices.csv",index=False)

In [7]:
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()
colors = px.colors.qualitative.Plotly
df3 = pd.read_csv("VideoCardsPrices.csv")

lista = df3.columns.tolist()

cols_to_delete = df3.columns[df3.isnull().sum()/len(df3) > .10]

lista_final=[x for x in lista if x not in cols_to_delete]

lista_final3060 = [x for x in lista if x in df3.columns[df3.columns.str.contains('RTX 3060')]]
lista_final3070 = [x for x in lista if x in df3.columns[df3.columns.str.contains('RTX 3070')]]
lista_final3080 = [x for x in lista if x in df3.columns[df3.columns.str.contains('RTX 3080')]]
lista_final3090 = [x for x in lista if x in df3.columns[df3.columns.str.contains('RTX 3090')]]
lista_finalGTX = [x for x in lista if x in df3.columns[df3.columns.str.contains('GTX')]]
lista_delete = lista_final3060 + lista_final3070 + lista_final3080 +lista_final3090 + lista_finalGTX
lista_ramase = [x for x in lista if x not in lista_delete]

#Setari butoane
um = [ {} for _ in range(len(df3.columns)) ]
buttons = []
menuadjustment = 0.15

buttonX = -0.1
buttonY = 1 + menuadjustment

li = []
li2 = []
li3 = []
li4 = []
li5 = []
li6 = []
li7 = []

# BUTON ALL
for i in range(1,len(df3.columns),1):
    fig.add_trace(go.Scatter(
    x= df3['Data curenta'],
    y=df3.iloc[:,i],
    name = df3.columns[i]
))

button2 = dict(method='restyle',
               label='All',
               visible=True,
               args=[{'visible':True}],
               args2 = [{'visible': False}],
               )
# assign button2 to an updatemenu and make some adjustments
um.append(dict())
um[i+1]['buttons'] = [button2]
um[i+1]['showactive'] = False
um[i+1]['y']=buttonY - 0.15
um[i+1]['x'] = buttonX

#BUTON PUTINE
for i in range(1,len(lista_final),1):
    li.append(df3.columns.get_loc(lista_final[i])-1)
   
    fig.add_trace(go.Scatter(
        x= df3['Data curenta'],
        y=df3.loc[:,lista_final[i]],
        name = lista_final[i]
        ))


button = dict(method='restyle',
                  label="Putine",
                  visible=True,
                  args=[{'visible':True,
                         'line.color' : colors[:len(li)]}, li],
                  args2 = [{'visible': False,
                            'line.color' : colors[:len(li)]}, li],
                 )
    
    # adjust some button features
buttonY = buttonY-menuadjustment
um[2]['buttons'] = [button]
um[2]['showactive'] = False
um[2]['y'] = buttonY - 0.85
um[2]['x'] = buttonX


#AL TREILEA BUTON    
for i in range(0,len(lista_final3060),1):
    li2.append(df3.columns.get_loc(lista_final3060[i])-1)

    fig.add_trace(go.Scatter(
        x= df3['Data curenta'],
        y=df3.loc[:,lista_final3060[i]],
        name = lista_final3060[i]
        ))
    

button3 = dict(method='restyle',
                  label="3060",
                  visible=True,
                  args=[{'visible':True,
                         'line.color' : colors[:len(li2)]}, li2],
                  args2 = [{'visible': False,
                            'line.color' : colors[:len(li2)]}, li2],
                 )


um.append(dict())
um[3]['buttons'] = [button3]
um[3]['showactive'] = False
um[3]['y']=buttonY-0.25
um[3]['x'] = buttonX

#AL PATRULEA BUTON
for i in range(0,len(lista_final3070),1):
    li3.append(df3.columns.get_loc(lista_final3070[i])-1)

    fig.add_trace(go.Scatter(
        x= df3['Data curenta'],
        y=df3.loc[:,lista_final3070[i]],
        name = lista_final3070[i]
        ))
    

button4 = dict(method='restyle',
                  label="3070",
                  visible=True,
                  args=[{'visible':True,
                         'line.color' : colors[:len(li3)]}, li3],
                  args2 = [{'visible': False,
                            'line.color' : colors[:len(li3)]}, li3],
                 )


um.append(dict())
um[4]['buttons'] = [button4]
um[4]['showactive'] = False
um[4]['y']=buttonY-0.35
um[4]['x'] = buttonX

#AL CINCILEA BUTON
for i in range(0,len(lista_final3080),1):
    li4.append(df3.columns.get_loc(lista_final3080[i])-1)

    fig.add_trace(go.Scatter(
        x= df3['Data curenta'],
        y=df3.loc[:,lista_final3080[i]],
        name = lista_final3080[i]
        ))


button5 = dict(method='restyle',
                  label="3080",
                  visible=True,
                  args=[{'visible':True,
                         'line.color' : colors[:len(li4)]}, li4],
                  args2 = [{'visible': False,
                            'line.color' : colors[:len(li4)]}, li4],
                 )


um.append(dict())
um[5]['buttons'] = [button5]
um[5]['showactive'] = False
um[5]['y']=buttonY-0.45
um[5]['x'] = buttonX

#AL SASELEA BUTON
for i in range(0,len(lista_final3090),1):
    li5.append(df3.columns.get_loc(lista_final3090[i])-1)

    fig.add_trace(go.Scatter(
        x= df3['Data curenta'],
        y=df3.loc[:,lista_final3090[i]],
        name = lista_final3090[i]
        ))
 

button6 = dict(method='restyle',
                  label="3090",
                  visible=True,
                  args=[{'visible':True,
                         'line.color' : colors[:len(li5)]}, li5],
                  args2 = [{'visible': False,
                            'line.color' : colors[:len(li5)]}, li5],
                 )


um.append(dict())
um[6]['buttons'] = [button6]
um[6]['showactive'] = False
um[6]['y']=buttonY-0.55
um[6]['x'] = buttonX

#AL SAPTELEA BUTON
for i in range(0,len(lista_finalGTX),1):
    li6.append(df3.columns.get_loc(lista_finalGTX[i])-1)

    fig.add_trace(go.Scatter(
        x= df3['Data curenta'],
        y=df3.loc[:,lista_finalGTX[i]],
        name = lista_finalGTX[i]
        ))
 

button7 = dict(method='restyle',
                  label="GTX",
                  visible=True,
                  args=[{'visible':True,
                         'line.color' : colors[:len(li6)]}, li6],
                  args2 = [{'visible': False,
                            'line.color' : colors[:len(li6)]}, li6],
                 )


um.append(dict())
um[7]['buttons'] = [button7]
um[7]['showactive'] = False
um[7]['y']=buttonY-0.65
um[7]['x'] = buttonX

#AL OPTULEA BUTON
for i in range(1,len(lista_ramase),1):
    li7.append(df3.columns.get_loc(lista_ramase[i])-2)

    fig.add_trace(go.Scatter(
        x= df3['Data curenta'],
        y=df3.loc[:,lista_ramase[i]],
        name = lista_ramase[i]
        ))


button8 = dict(method='restyle',
                  label="Others",
                  visible=True,
                  args=[{'visible':True,
                         'line.color' : colors[:len(li7)]}, li7],
                  args2 = [{'visible': False,
                            'line.color' : colors[:len(li7)]}, li7],
                 )


um.append(dict())
um[8]['buttons'] = [button8]
um[8]['showactive'] = False
um[8]['y']=buttonY-0.75
um[8]['x'] = buttonX

# add dropdown menus to the figure
fig.update_layout(showlegend=True, updatemenus=um)

# adjust button type
for m in fig.layout.updatemenus:
    m['type'] = 'buttons'

fig.show()


In [8]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
from pprint import pprint

Filter_df = pd.read_csv('AnalizaPlaci2.csv',index_col=None)
Filter_df = Filter_df.drop('Ventilatoare',axis=1)
Filter_df = Filter_df[2:].replace(" ","")
Filter_df['Data curenta'] = pd.to_datetime(Filter_df['Data curenta'],dayfirst=True)
df3 =Filter_df.groupby(['Data curenta','Chipset']).mean().unstack().reset_index()


lista1= []
listanp = df3.columns.to_numpy()
lista1 = np.append(lista1,listanp[0][0])

listanp = np.delete(listanp,0)
for i in listanp:
    lista1 = np.append(lista1, i[1])


df3.columns = lista1
df3.to_csv("ChipsetPrices.csv",index=False)

In [9]:
import plotly.express as px
import pandas as pd 

df3 = pd.read_csv("ChipsetPrices.csv")

lista = df3.columns.tolist()
cols_to_delete = df3.columns[df3.tail(1).isnull().sum() > 0]
lista_final=[x for x in lista if x not in cols_to_delete]

fig = px.bar(x=lista_final[1:], y=df3.tail(1).loc[:,lista_final[1:]].values.tolist()[0],
                title='Pret Chipset in medie',labels={'x':'Chipset','y':'Pirce'})
fig.show()

In [10]:
import pandas as pd 
from IPython.display import display
df_daily = pd.read_csv("AnalizaPlaci2.csv")
data_curenta = df_daily['Data curenta'].iloc[-1]
#data_aleasa = input('Alege data(format: dd/mm/yyyy)')
df_daily = df_daily.loc[df_daily['Data curenta'] == data_curenta]
df_daily = df_daily[['Data curenta','Chipset','Denumire Produs','Ventilatoare','Pret LEI','Site','Sursa']]
df_daily.rename(columns={'Data curenta' : 'Data'},inplace=True)
df_daily = df_daily[df_daily['Denumire Produs'].str.contains('3060')]
display(df_daily.sort_values('Pret LEI',ascending=False))

,Data,Chipset,Denumire Produs,Ventilatoare,Pret LEI,Site,Sursa
3123,09/02/2022,RTX3060,GIGABYTE GeForce RTX 3060 GAMING OC 12GB GDD...,3.0,5053.20,ipon.ro,https://www.compari.ro/Jump.php?ProductAttribu...
3113,09/02/2022,RTX3060Ti,GIGABYTE GeForce RTX 3060 Ti 8GB GDDR6 256bi...,3.0,4492.25,devodep.ro,https://www.compari.ro/Jump.php?ProductAttribu...
3120,09/02/2022,RTX3060Ti,GIGABYTE GeForce RTX 3060 Ti GAMING OC 8GB G...,3.0,4390.34,forit.ro,https://www.compari.ro/Jump.php?ProductAttribu...
3103,09/02/2022,RTX3060Ti,MSI GeForce RTX 3060 Ti 8GB GDDR6 (RTX 3060 ...,2.0,4298.99,forit.ro,https://www.compari.ro/Jump.php?ProductAttribu...
3115,09/02/2022,RTX3060Ti,ZOTAC GeForce GAMING Twin Edge GeForce RTX 3...,2.0,4144.50,leumart.ro,https://www.compari.ro/Jump.php?ProductAttribu...
3117,09/02/2022,RTX3060,MSI GeForce RTX 3060 X 12GB GDRR6 192bit (RT...,2.0,3799.00,cel.ro,https://www.compari.ro/Jump.php?ProductAttribu...
3108,09/02/2022,RTX3060,ZOTAC GeForce GAMING RTX 3060 12GB GDDR6 192...,2.0,3605.50,leumart.ro,https://www.compari.ro/Jump.php?ProductAttribu...
3119,09/02/2022,RTX3060,GIGABYTE GeForce RTX 3060 12GB GDDR6 192bit ...,2.0,3599.99,forit.ro,https://www.compari.ro/Jump.php?ProductAttribu...
3118,09/02/2022,RTX3060,ZOTAC GeForce RTX 3060 Twin Edge OC 12GB GDD...,2.0,3305.23,devodep.ro,https://www.compari.ro/Jump.php?ProductAttribu...
3106,09/02/2022,RTX3060,GIGABYTE GeForce RTX 3060 12GB GDDR6 192bit ...,3.0,3069.00,cumperi-orice.ro,https://www.compari.ro/Jump.php?ProductAttribu...


In [11]:
import pandas as pd
from IPython.display import display

df_OutOfStock = pd.read_csv('VideoCardsPrices.csv').drop('Data curenta',axis=1)
df_OutOfStock.columns = [x[0] for x in df_OutOfStock.columns.str.split("(",1).tolist()]
df_out = pd.DataFrame()
for i in range(0,len(df_OutOfStock.columns)):
    df_out = df_out.append(df_OutOfStock.iloc[::-1,i].isnull().astype(int).groupby(df_OutOfStock.iloc[::-1,i].notnull().astype(int).cumsum()).sum().iloc[:1],ignore_index=False)
df_out.drop(1,axis=1,inplace=True)
df_out.dropna(axis=0,inplace=True)
df_out.rename(columns={0:'Zile out of stock'},inplace=True)
df_out = df_out.astype({'Zile out of stock':'int16'})
df_out = df_out[df_out.index.str.contains('6900')]
display(df_out)



C:\Users\Dumbri\AppData\Local\Temp\ipykernel_20044\4250839752.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Dumbri\AppData\Local\Temp\ipykernel_20044\4250839752.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Dumbri\AppData\Local\Temp\ipykernel_20044\4250839752.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Dumbri\AppData\Local\Temp\ipykernel_20044\4250839752.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Dumbri\AppData\Local\Temp\ipykernel_20044\4250839752.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

,Zile out of stock
PowerColor Radeon Red Devil Ultimate RX 6900 XT 16GB GDDR6 256bit,23
SAPPHIRE Radeon NITRO+ SE RX 6900 XT 16GB DDR6 256bit,9
XFX Radeon Speedster MERC 319 Black RX 6900 XT 16GB GDDR6 256bit,9
